In [9]:
import ogr
import subprocess

import geopandas as gpd

Reproject a shape file

In [22]:
!ogr2ogr ../scratch/deelbekkens_wgs84 -t_srs "EPSG:4326" ../data/deelbekkens/Deelbekken.shp

In [ ]:
# TODO: add creation of subcat met geopandas!

Single clip

voorbeeld ahv -> http://www.gdal.org/gdalwarp.html

**DOEL**: 1 deelbekken van de nete uitsnijden uit de tif:

In [7]:
import subprocess

inraster = '../scratch/NE1_50M_SR/NE1_50M_SR.tif'
outraster = inraster.replace('.tif', '{}.tif'.format("_out"))    
inshape = "../scratch/subcat.shp"
subprocess.call(['gdalwarp', inraster, outraster, '-cutline', inshape, 
                     '-crop_to_cutline', '-overwrite'])

0

**DOEL**: provincie op naam opgeven en die uitsnijden

In [10]:
provincies = gpd.read_file("../scratch/provinciebel.shp")
provincies

,GEWEST,HECTARES,Naam,OBJECTID,PROVINCIE,Shape_Area,Shape_Leng,geometry
0,Waals,445977.11,Luxemburg,1,LUXEMBURG,4.459771e+09,526369.525162,"POLYGON ((268274.9058406549 98186.10143288126,..."
1,Waals,367595.26,Namen,2,NAMEN,3.675953e+09,549687.828645,"POLYGON ((192630.9060262059 148439.2967436832,..."
2,Waals,109697.28,Waals-Brabant,3,WAALS-BRABANT,1.096973e+09,282779.214685,"POLYGON ((180156.0157745878 165085.0936231858,..."
3,Waals,381311.70,Henegouwen,4,HENEGOUWEN,3.813117e+09,661468.239273,(POLYGON ((54813.49533785344 163427.0036376416...
4,Waals,385673.69,Luik,5,LUIK,3.856737e+09,516619.459484,"POLYGON ((242820.1453689483 161318.1724036776,..."
5,Vlaams,287618.61,Antwerpen,6,ANTWERPEN,2.876186e+09,409253.016619,(POLYGON ((189455.0002312482 233439.9997988463...
6,Vlaams,242796.85,Limburg,7,LIMBURG,2.427968e+09,396268.042066,(POLYGON ((249425.7968311003 164017.6248221917...
7,Vlaams,300793.98,Oost-Vlaanderen,8,OOST-VLAANDEREN,3.007940e+09,404086.523154,"POLYGON ((142473.9375314937 226522.1563364035,..."
8,Vlaams,211881.34,Vlaams-Brabant,9,VLAAMS-BRABANT,2.118813e+09,506036.167038,"POLYGON ((200518.4063215517 193496.9689776464,..."
9,Vlaams,319337.75,West-Vlaanderen,10,WEST-VLAANDEREN,3.193377e+09,355831.654428,"POLYGON ((80027.18726533552 229580.6088192519,..."


In [11]:
inraster = '../scratch/NE1_50M_SR/NE1_50M_SR.tif'
outraster = inraster.replace('.tif', '{}.tif'.format("_OostVlaanderen"))    
inshape = "../scratch/provinciebel.shp"
subprocess.call(['gdalwarp', inraster, outraster, '-cutline', inshape, 
                 '-cwhere', "PROVINCIE='OOST-VLAANDEREN'", 
                 '-crop_to_cutline', 
                 '-overwrite'])

0

Maar ik wil dat liever automatisch doen voor alle individuele provincies en voor elk van die een geotiff maken:

In [12]:
import ogr

inraster = '../scratch/NE1_50M_SR/NE1_50M_SR.tif'
outraster = inraster.replace('.tif', '{}.tif'.format("_OostVlaanderen"))    
inshape = "../scratch/provinciebel.shp"

ds = ogr.Open(inshape)
lyr = ds.GetLayer(0)

lyr.ResetReading()
ft = lyr.GetNextFeature()

while ft:

    province_name = ft.GetFieldAsString('PROVINCIE')
    print(province_name)

    outraster = inraster.replace('.tif', '_%s.tif' % province_name.replace('-', '_'))    
    subprocess.call(['gdalwarp', inraster, outraster, '-cutline', inshape, 
                     '-crop_to_cutline', '-cwhere', "PROVINCIE='%s'" %province_name])

    ft = lyr.GetNextFeature()

ds = None

LUXEMBURG
NAMEN
WAALS-BRABANT
HENEGOUWEN
LUIK
ANTWERPEN
LIMBURG
OOST-VLAANDEREN
VLAAMS-BRABANT
WEST-VLAANDEREN
BRUSSEL


To gain from my efforts, a reusable function would be what I could really use